In [6]:
import math

class DGIM:
    def __init__(self, k):
        self.k = k
        self.buckets = []

    def add(self, bit):
        if bit != 1:
            return
        
        current_time = len(self.buckets)
        
        # Merge any buckets of the same size to maintain the invariant.
        while len(self.buckets) >= 2 and self.buckets[-2][0] == self.buckets[-1][0]:
            self.buckets[-2] = (self.buckets[-2][0] * 2, self.buckets[-2][1] + self.buckets[-1][1])
            self.buckets.pop()
        
        # Create a new bucket with size 1.
        self.buckets.append((1, 1))
        
        # Discard any buckets that are older than k.
        while len(self.buckets) > 1 and current_time - self.buckets[0][0] >= self.k:
            self.buckets.pop(0)
    
    def count_ones_before_k(self):
        count = 0
        
        # Traverse the buckets from the newest to the oldest.
        for i in range(len(self.buckets) - 1, -1, -1):
            size, ones = self.buckets[i]
            if size <= self.k:
                count += ones
            else:
                # Split the bucket into two smaller buckets.
                count += (size // 2) * (i + 1)
                break
                
        return count
    
# Example usage:
stream = [0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1]
k = 4

dgim = DGIM(k)
for bit in stream:
    dgim.add(bit)
    
count = dgim.count_ones_before_k()
print(count)  


7


In [8]:
!pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5976 sha256=6ef33ff79153881b9d74e5a349ae062c4abe161c51d52b87a7498cb09a96f8ff
  Stored in directory: /root/.cache/pip/wheels/32/2a/54/10c595515f385f3726642b10c60bf788029e8f3a1323e3913a
Successfully built apyori


In [9]:
from apyori import apriori

def stream_apriori(stream, min_support):
    window_size = 100  # Size of sliding window
    window = []
    
    for item in stream:
        window.append(item)
        if len(window) >= window_size:
            results = apriori(window, min_support=min_support)
            for itemset in results:
                print(itemset.items, itemset.support)
            window.pop(0)
            
    # Process any remaining transactions in the window
    if len(window) > 0:
        results = apriori(window, min_support=min_support)
        for itemset in results:
            print(itemset.items, itemset.support)


In [10]:
from apyori import apriori

# Sample stream of transactions
stream = [    ['A', 'B', 'C'],
    ['B', 'C', 'D'],
    ['A', 'C', 'E'],
    ['B', 'C', 'E'],
    ['A', 'B', 'C', 'E'],
    ['A', 'B', 'D', 'E'],
    ['B', 'C', 'E'],
    ['A', 'B', 'C', 'E'],
    ['A', 'B', 'D', 'E'],
    ['A', 'B', 'C', 'E'],
]

# Minimum support threshold
min_support = 0.2

stream_apriori(stream, min_support)


frozenset({'A'}) 0.7
frozenset({'B'}) 0.9
frozenset({'C'}) 0.8
frozenset({'D'}) 0.3
frozenset({'E'}) 0.8
frozenset({'B', 'A'}) 0.6
frozenset({'A', 'C'}) 0.5
frozenset({'D', 'A'}) 0.2
frozenset({'A', 'E'}) 0.6
frozenset({'B', 'C'}) 0.7
frozenset({'D', 'B'}) 0.3
frozenset({'B', 'E'}) 0.7
frozenset({'C', 'E'}) 0.6
frozenset({'D', 'E'}) 0.2
frozenset({'B', 'A', 'C'}) 0.4
frozenset({'D', 'B', 'A'}) 0.2
frozenset({'B', 'A', 'E'}) 0.5
frozenset({'A', 'C', 'E'}) 0.4
frozenset({'D', 'A', 'E'}) 0.2
frozenset({'B', 'C', 'E'}) 0.5
frozenset({'D', 'B', 'E'}) 0.2
frozenset({'B', 'A', 'C', 'E'}) 0.3
frozenset({'D', 'B', 'A', 'E'}) 0.2


In [13]:
!pip install mmh3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import mmh3
import heapq

class TrendingHashtags:
    def __init__(self, width, depth, threshold):
        self.width = width
        self.depth = depth
        self.threshold = threshold
        self.count_min = [[0] * width for _ in range(depth)]
        self.heap = []
        
    def update(self, hashtag):
        for i in range(self.depth):
            pos = mmh3.hash(hashtag, i) % self.width
            self.count_min[i][pos] += 1
            
        if hashtag in [x[1] for x in self.heap]:
            self.heap = [(x[0] + 1, x[1]) if x[1] == hashtag else x for x in self.heap]
        else:
            heapq.heappush(self.heap, (1, hashtag))
            
        while self.heap and self.heap[0][0] + self.threshold <= self.count(hashtag):
            heapq.heappop(self.heap)
            
    def count(self, hashtag):
        return min([self.count_min[i][mmh3.hash(hashtag, i) % self.width] for i in range(self.depth)])
    
    def get_trending(self):
        if not self.heap:
            return None
        return self.heap[0][1]

# Example usage
trending = TrendingHashtags(width=100, depth=5, threshold=10)
hashtags = ['#hello', '#world', '#hello', '#trending', '#hello', '#world', '#world', '#trending', '#world', '#trending', '#world', '#world', '#world', '#trending', '#world', '#trending', '#trending', '#trending', '#trending']

for hashtag in hashtags:
    trending.update(hashtag)

print(trending.get_trending())  # Output: '#world'


#trending
